# Checkpoint 1 - Setting working directory and combining complete and incomplete responses

## Preliminary steps:
1. The raw data file has been prepared to facilate importing and concatenation across spread sheets (see 'raw_data'). The raw data with the Key retained is saved as raw_data_key.

## Importing and cleaning
Once the data are imported into a pandas dataframe, the following steps will be taken:
1. Concantenation across spreadsheets to combine completed and non-completed responses (column names and length aligned)
2. Re-structuring of dataframe to include only those variables of interest (i.e. removing text responses questions) and remove composite scores, as these will need to be re-calculated after the missing data analysis. 
3. Recoding of missing data values (initially set as 9999) to a value that SPSS/R pacakges can handle


### Check and set CWD

In [1]:
import os

#check and set working directory
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))
print(cwd+ "\\" +files[0])

os.chdir('c:\\Users\\j_m289\\Pictures\\PHD\\15. Data Analysis\HMR\Raw')  # Provide the new path here

#try to use relative path for files (tied to directory and nesting within folders)
path = "/Users/j_m289\Pictures\PHD\15. Data Analysis\HMR\Raw"
  
start = "/Users/j_m289"

relative_path = os.path.relpath(path, start)

#Load packages
import pandas as pd #pandas for data frame management and descriptives (in addition to numpy)
import seaborn as sns #seaborn for visualisation (MATPLOTLIB also)
import scipy as sci #scipy for HMR model
import numpy as np
import matplotlib.pyplot as plt
import missingno as mn
import miceforest as mf
from sklearn.impute import SimpleImputer
from scipy import stats

Files in 'c:\\Users\\j_m289\\Pictures\\PHD\\15. Data Analysis\\HMR\\Code': ['1. Raw Data Combining and Preliminary Clean.ipynb', '2. Missing Data Analysis.ipynb', '3. File Splitting.ipynb', '4. Outlier Detection (version 2).ipynb', '4. Outlier Detection.ipynb', '5. Descriptives.ipynb', '5. Hierarchical Regression.ipynb']
c:\Users\j_m289\Pictures\PHD\15. Data Analysis\HMR\Code\1. Raw Data Combining and Preliminary Clean.ipynb


### Load Packages and read file into pandas df

In [ ]:
# missing values list is added so that pandas can read these
missing_values =['n/a', 'na', '9999'] 

#read excel file with pandas (note that both sheets are read separately) assigning each sheet to separate df
df_raw_comp = pd.read_excel('raw_data.xlsx', sheet_name= 'Completed', na_values = missing_values, engine='openpyxl') # all complete survey responses
df_raw_comp.set_index(['ID']) # Index set to aid data registration

df_raw_incomp = pd.read_excel('raw_data.xlsx', sheet_name= 'Incomplete', na_values = missing_values, engine='openpyxl')# all incomplete survey responses

### Concantenate excel sheets

In [ ]:
#Prior to combining the datasets, need to determine the columns that differ....

#The following compares the columns in the complete and incomplete datasets:
#       1. Assigns the difference to the variable column diff
#       2. Parses the difference lists to the .drop function 
#       3. Prints the difference in lists to check
#       4. Drops nuisance rows that contain formatting data not needed from both datasets

print(list(set(df_raw_comp.columns) - set(df_raw_incomp.columns)))
column_diff = list(set(df_raw_comp.columns) - set(df_raw_incomp.columns))
"""***"""

df_raw_comp.drop(columns=column_diff, axis = 1, inplace=True) # axis = 1 for columns (0= rows by default)
print(list(set(df_raw_comp.columns) - set(df_raw_incomp.columns)))


df_raw_incomp.drop(index = [0,1,158,159,160], axis = 0, inplace=True)# four rows dropped
df_raw_comp.drop(index = [0,1], axis = 0, inplace=True)# two rows dropped

#The completed and incomplete datasets are then combined, assigning a key to verify source.
frames = [df_raw_comp, df_raw_incomp]
df_concan = pd.concat(frames, ignore_index=True)

display(df_concan) # visual inspection

### Preliminary variable cleaning

In [ ]:
#The following drops rows and columns not needed for analysis and renames columns.
df_raw_clean = df_concan.drop(columns= ['Duration (in minutes)','RecordedDate','BMI_low','Nationality_7_TEXT','Diagnosis_type_13_TEXT','Eating_issue_type',\
    'ED_Diagnosis_GROUPED','ED_Recovered','Treated_ED','EF_14F','S_14F','EC_14F','BHS_O_14F','EOS_14F','SC_14F',\
    'HB_DS_14F','SB_14F','HSC_SD_14F','RE_14F','SD_14F','HIS_SC_14F','SA_14F','HSR_C_14F','EF_Mean_14F','S_Mean_14F',\
    'EC_Mean_14F','BHS_O_Mean_14F','EOS_Mean_14F','SC_Mean_14F','HB_DS_Mean_14F',
    'SB_Mean_14F','HSC_SD_Mean_14F','RE_Mean_14F','SD_Mean_14F','HIS_SC_Mean_14F','SA_Mean_14F',	
    'HSR_C_Mean_14F','EFSA_9F','S_9F','EC_9F','O_9F','EOS_9F','SCRE_9F','DS_9F','SB_9F','HSC_9F','EF_9F_Mean',\
    'S_9F_Mean','EC_9F_Mean','BHS_O_9F_Mean','EOS_9F_Mean','SC_9F_Mean','HB_DS_9F_Mean','SB_9F_Mean',\
    'HSC_SD_9F_Mean','EDEQ_R','EDEQ_EC','EDEQ_SC','EDEQ_WC','EDEQ_GS',\
    'EDEQ_Clin_Ct_1','EDEQ_Clin_Ct_2.1', 'EDEQ_Clin_Ct_2.2',\
    'EDEQ_Clin_Ct_3','EDEQ_Clin_Ct_4a_BN','EDEQ_Clin_Ct_4b_AN','EDEQ_Clin_Ct_4c_BED','ED_Sub_Group',\
    'EDEQ_GS_NClin_Ct','Appearance_Satisfact_NPS_GROUP','CFI_2_R','CFI_4_R','CFI_7_R','CFI_9_R','CFI_11_R',\
    'CFI_17_R','CFI_Alt','CFI_Ctrl','CFI_Total','CFI_Additioal_Total','ERQ_Cognitive_Reappraisal_Total',
    'ERQ_Expressive_Suppression_Total','ERQ_Cognitive_Reappraisal_Average',	'ERQ_Expressive_Suppression_Average',\
    'Unnamed: 24', 'Unnamed: 25'
], inplace=False) #Use inplace = true to update dataframe.

# The following renames columns 
df_raw_clean = df_raw_clean.rename(columns={'Diagnosis': 'MH_Diag','Diagnosis_type': 'MH_Diag_type', 'Q29': 'ED_Diag',\
    'ED_Diagnosis': 'ED_subtype', 'Q28': 'past_ED_Diag', 'Q30': 'past_ED_subtype', 'Eating_Treatment': 'ED_treatment',\
    'CFI-additional_1':'CFI-add1','CFI-additional_2':'CFI-add2','CFI-additional_3':'CFI-add3','CFI-additional_4':'CFI-add4',\
    'CFI-additional_5':'CFI-add5','CFI-additional_6':'CFI-add6','CFI-additional_7':'CFI-add7','CFI-additional_8':'CFI-add8',\
    'Emotion_Regulation_1':'EmReg_1','Emotion_Regulation_2':'EmReg_2','Emotion_Regulation_3':'EmReg_3','Emotion_Regulation_4':'EmReg_4',\
    'Emotion_Regulation_5':'EmReg_5','Emotion_Regulation_6':'EmReg_6','Emotion_Regulation_7':'EmReg_7','Emotion_Regulation_8':'EmReg_8',\
    'Emotion_Regulation_9':'EmReg_9','Emotion_Regulation_10':'EmReg_10'
    }, errors="raise")

### Assigns values to variables where non-response indicates no or Non-applicable question ###
values = {"MH_Diag_type": 0, "ED_Diag": 2, "ED_subtype": 0, "past_ED_Diag": 2, "past_ED_subtype": 0}
df_mis = df_raw_clean.fillna(value=values)

### the following checks if values added are consisent with key ###
total_EDdiag = df_mis['ED_Diag'].value_counts()[2] + df_mis['ED_Diag'].value_counts()[3]# The following check combines the cases for ED_Diag of 2 (= No diagnosis) and 3 (Prefer to to say)
print("The amount of cases where 0 is incorrectly put for ED_subtype is " \
+ str(total_EDdiag - df_mis['ED_subtype'].value_counts()[0]) + " %")

total_pastEDdiag = df_mis['past_ED_Diag'].value_counts()[2] + df_mis['past_ED_Diag'].value_counts()[3]# The following check combines the cases for past_ED_Diag of 2 (= No diagnosis) and 3 (Prefer to to say)
print("The amount of cases where 0 is incorrectly put for past_ED_subtype is " \
+ str(total_pastEDdiag - df_mis['past_ED_subtype'].value_counts()[0]) + " %")

###Export to xlsx file in CWD###
df_mis.to_excel('raw_clean.xlsx', index=True)

### At this point the file was visually inspected and imported in SPSS for missing data analysis.
- Little MCAR calculation 